In [ ]:
!nvidia-smi

Mon May 30 06:15:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installs

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install arcgis==1.8.5.post3
!pip install transformers==3.3.0
!pip install imblearn

In [19]:
akzo_gdrive = '/content/gdrive/MyDrive/akzo_models/Additives_ner_ktp_specific_test'

# Imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
!git clone https://github.com/akhilnegi/temp_del.git temp_del1
!cp /content/temp_del1/ner_file/_ner_spacy.py  /usr/local/lib/python3.7/dist-packages/arcgis/learn/text/_ner_spacy.py
!cp /content/temp_del1/point_cnn_file/_pointcnn_utils.py  /usr/local/lib/python3.7/dist-packages/arcgis/learn/models/_pointcnn_utils.py



Cloning into 'temp_del1'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 100 (delta 34), reused 80 (delta 17), pack-reused 0
Receiving objects: 100% (100/100), 31.39 MiB | 28.47 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [4]:
from arcgis.learn.text import TextClassifier, EntityRecognizer
from arcgis.learn import prepare_data, prepare_textdata
import json, os
import pandas as pd

/usr/local/lib/python3.7/dist-packages/torchvision/models/segmentation/segmentation.py:8: UserWarning: The 'torchvision.models.segmentation.segmentation' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.models.segmentation' directly instead.
  "The 'torchvision.models.segmentation.segmentation' module is deprecated since 0.12 and will be removed in "


# NER for extracting TDS properties

## Preparing data for arcgis NER

In [5]:
df = pd.read_excel('/content/temp_del1/additives/stratified_batch/additives_sample_data_classification.xlsx')

In [6]:
df.head(1)

,filename,from,pn,x_min,y_min,x_max,y_max,large_doc,class,classification_text,ner_text,jsonl_data
0,1C002 TDS Avebe 010910.json,table0,0,1.4207,3.3967,7.1372,3.6948,NaN,irrelevant,Requirements |,Requirements |,"{'data': 'Requirements |', 'label': []}"


In [7]:
df['class_'] = df['class']
df['label'] = df['class']

In [8]:
df = df[df.label.apply(lambda x: x in ['viscosity', 'particle_size', 'density', 'color', 'ph', 'solid_content',
       'hydroxyl_value', 'acid_content', 'cas_number', 'active_%', 'stray', 'water_miscibility', 'multi',
       'molecular_weight', 'flash_point', 'boiling_point', 'absorption_wavelength'])]

In [9]:
df.label.unique()

array(['viscosity', 'particle_size', 'density', 'color', 'ph', 'solid_content', 'hydroxyl_value', 'acid_content',
       'cas_number', 'active_%', 'stray', 'water_miscibility', 'multi', 'molecular_weight', 'flash_point',
       'boiling_point', 'absorption_wavelength'], dtype=object)

In [10]:
from collections import Counter
Counter(df.label)

Counter({'absorption_wavelength': 5,
         'acid_content': 115,
         'active_%': 104,
         'boiling_point': 37,
         'cas_number': 99,
         'color': 473,
         'density': 466,
         'flash_point': 90,
         'hydroxyl_value': 32,
         'molecular_weight': 66,
         'multi': 34,
         'particle_size': 203,
         'ph': 152,
         'solid_content': 91,
         'stray': 107,
         'viscosity': 271,
         'water_miscibility': 44})

In [11]:
from imblearn.over_sampling import  RandomOverSampler
import numpy as np
import random

class_oversampling = {}
for label in df.label.unique():
  if label.lower() in ['irrelevant', 'header', 'multi', 'multi_header', 'measuring_condition']:
    continue
  class_oversampling[label.lower()]=271
class_oversampling = {'absorption_wavelength': 100,
         'acid_content': 115,
         'active_%': 104,
         'boiling_point': 100,
         'cas_number': 99,
         'color': 473,
         'density': 466,
         'flash_point': 90,
         'hydroxyl_value': 100,
         'molecular_weight': 66,
         'multi': 100,
         'particle_size': 203,
         'ph': 152,
         'solid_content': 91,
         'stray': 107,
         'viscosity': 271,
         'water_miscibility': 100}
oversample = RandomOverSampler(sampling_strategy=class_oversampling, random_state=1)

In [12]:
#oversampling data
X_over, y_over = oversample.fit_resample(np.array(df.jsonl_data.to_list()).reshape(-1,1), df.label.apply(str.lower).to_list())

In [13]:
df_oversampled_ner = pd.DataFrame(columns=['jsonl_data','class_'])
df_oversampled_ner.jsonl_data = X_over.reshape(-1)
df_oversampled_ner.class_ = y_over

In [14]:
df_oversampled_ner.class_.unique()

array(['viscosity', 'particle_size', 'density', 'color', 'ph', 'solid_content', 'hydroxyl_value', 'acid_content',
       'cas_number', 'active_%', 'stray', 'water_miscibility', 'multi', 'molecular_weight', 'flash_point',
       'boiling_point', 'absorption_wavelength'], dtype=object)

In [15]:
df_oversampled_ner[df_oversampled_ner.class_=='absorption_wavelength']

,jsonl_data,class_
1129,{'data': '1.3 | Light Absorbance | Ph. Eur. ( ...,absorption_wavelength
1326,{'data': ' | DOP absorption ( ISO 787 / 5 ) | ...,absorption_wavelength
2106,{'data': 'Absorption ( nm ) : | 252 - 325 | 25...,absorption_wavelength
2114,"{'data': 'Absorption: | 248 nm', 'label': [[14...",absorption_wavelength
2140,{'data': 'Transmittance @ 500 nm | Spectrophot...,absorption_wavelength
...,...,...
2479,{'data': 'Transmittance @ 500 nm | Spectrophot...,absorption_wavelength
2480,{'data': 'Transmittance @ 500 nm | Spectrophot...,absorption_wavelength
2481,{'data': ' | DOP absorption ( ISO 787 / 5 ) | ...,absorption_wavelength
2482,{'data': '1.3 | Light Absorbance | Ph. Eur. ( ...,absorption_wavelength


## KTP based NER

In [17]:
for ktp in df_oversampled_ner.class_.unique():
  json_path = f'/content/{ktp}_jsonl_data_lm.jsonl'
  updated_json_path = f'/content/{ktp}_jsonl_data_lm.jsonl'
  with open(json_path,'a') as f:
    for tup in df_oversampled_ner[df_oversampled_ner.class_==ktp].itertuples():
          f.write(f'{tup.jsonl_data}\n')
  with open(json_path) as f:
    json_list = f.readlines()
  for i,json_txt in enumerate(json_list):
    json_dict = eval(json_txt.replace('\n',''))
    json_dict['text']=json_dict.pop('data')
    json_dict['labels'] = json_dict.pop('label')
    json_list[i] = json.dumps(json_dict)
  if os.path.exists(updated_json_path):
    os.remove(updated_json_path)
  for item in json_list:
      with open(updated_json_path, 'a') as f:
          f.write(item+'\n')

In [ ]:
import glob
for updated_json_path in glob.glob('/content/*.jsonl'):
  ktp = updated_json_path.split('_jsonl_data_lm.jsonl')[0].split('/')[-1]
  print(ktp)
  data = prepare_data(path=updated_json_path, dataset_type='ner_json')#, batch_size=1)
  model = EntityRecognizer(data)
  lr = model.lr_find()
  model.fit(epochs=30, lr=lr)
  model.save(f'{akzo_gdrive}/additives_strativied_ner_spacy_model_v1_{ktp}', compute_metrics=False)


